# Estabelece a absorção de demanda pelos aeroportos mapeados com base na matriz de origem-destino

In [2]:
import pandas as pd

In [3]:
matriz_od_completa = pd.read_csv('data/csv/matriz_od_completa.csv', delimiter = ',')

In [4]:
cat_influencia = pd.read_excel('data/excel/xlsx/cat_influencia_geral.xlsx', engine='openpyxl')

In [8]:
cat_absorcao = pd.read_excel('data/excel/xlsx/cat_absorcao_nacional.xlsx', engine='openpyxl')

In [6]:
piar_aeroportos = ['SBTE - TERESINA', 'SBPB - PARNAÍBA', 'SWKQ - SÃO RAIMUNDO NONATO','SBFZ - FORTALEZA','SNBR - BARREIRAS','SBSL - SÃO LUÍS','SBBR - BRASÍLIA']

In [10]:
def filtrar_aeroportos(df, aeroportos, cat_absorcao):
    # Mantém as três primeiras colunas como índice
    colunas_indice = df.columns[:3]
    
    # Filtra as colunas que correspondem ao id_aeroporto na lista 'aeroportos'
    df_filtrado = df.loc[:, df.columns.isin(aeroportos) | df.columns.isin(colunas_indice)]
    
    # Filtra as linhas onde todas as colunas a partir da quarta são iguais a 'M'
    colunas_para_verificar = df_filtrado.columns[3:]  # As colunas a partir da quarta
    
    # Filtra as linhas onde todos os valores dessas colunas são 'M'
    df_filtrado = df_filtrado[~df_filtrado[colunas_para_verificar].eq('M').all(axis=1)]
    
    # Substituir valores categóricos pelas médias equivalentes em cat_absorcao
    categoria_para_absorcao = dict(zip(cat_absorcao['categoria'], cat_absorcao['absorcao']))
    df_filtrado[colunas_para_verificar] = df_filtrado[colunas_para_verificar].replace(categoria_para_absorcao)
    
    # Substituir 'M' restantes por 0
    df_filtrado[colunas_para_verificar] = df_filtrado[colunas_para_verificar].replace('M', 0)
    
    # Retorna o novo DataFrame filtrado
    return df_filtrado

cat_influencia_piar = filtrar_aeroportos(cat_influencia, piar_aeroportos, cat_absorcao)

In [14]:
def criar_rotas():
    # Dados para cada rota
    dados = {
        'rota_a_cenario_1': {
            'origem': [
                'SBPB - PARNAÍBA', 'SBTE - TERESINA', 'SWKQ - SÃO RAIMUNDO NONATO',
                'SBBR - BRASÍLIA', 'SWKQ - SÃO RAIMUNDO NONATO', 'SBTE - TERESINA',
                'SBPB - PARNAÍBA', 'SBTE - TERESINA', 'SBPB - PARNAÍBA',
                'SBBR - BRASÍLIA', 'SBBR - BRASÍLIA', 'SWKQ - SÃO RAIMUNDO NONATO'
            ],
            'destino': [
                'SBTE - TERESINA', 'SWKQ - SÃO RAIMUNDO NONATO', 'SBBR - BRASÍLIA',
                'SWKQ - SÃO RAIMUNDO NONATO', 'SBTE - TERESINA', 'SBPB - PARNAÍBA',
                'SBBR - BRASÍLIA', 'SBBR - BRASÍLIA', 'SWKQ - SÃO RAIMUNDO NONATO',
                'SBPB - PARNAÍBA', 'SBTE - TERESINA', 'SBPB - PARNAÍBA'                
            ]
        },
        'rota_a_cenario_2': {
            'origem': [
                'SBPB - PARNAÍBA', 'SBTE - TERESINA', 'SNBR - BARREIRAS',
                'SBBR - BRASÍLIA', 'SNBR - BARREIRAS', 'SBTE - TERESINA',
                'SBPB - PARNAÍBA', 'SBTE - TERESINA', 'SBPB - PARNAÍBA',
                'SBBR - BRASÍLIA', 'SBBR - BRASÍLIA', 'SNBR - BARREIRAS'
            ],
            'destino': [
                'SBTE - TERESINA', 'SNBR - BARREIRAS', 'SBBR - BRASÍLIA',
                'SNBR - BARREIRAS', 'SBTE - TERESINA', 'SBPB - PARNAÍBA',
                'SBBR - BRASÍLIA', 'SBBR - BRASÍLIA', 'SNBR - BARREIRAS',
                'SBPB - PARNAÍBA', 'SBTE - TERESINA', 'SBPB - PARNAÍBA'  
            ]
        },
        'rota_b_cenario_1': {
            'origem': [
                'SBPB - PARNAÍBA', 'SBSL - SÃO LUÍS', 'SBTE - TERESINA',
                'SBFZ - FORTALEZA', 'SBTE - TERESINA', 'SBSL - SÃO LUÍS',
                'SBPB - PARNAÍBA', 'SBSL - SÃO LUÍS', 'SBTE - TERESINA',
                'SBFZ - FORTALEZA'
            ],
            'destino': [
                'SBSL - SÃO LUÍS', 'SBTE - TERESINA', 'SBFZ - FORTALEZA',
                'SBTE - TERESINA', 'SBSL - SÃO LUÍS', 'SBPB - PARNAÍBA',
                'SBTE - TERESINA', 'SBFZ - FORTALEZA', 'SBPB - PARNAÍBA',
                'SBSL - SÃO LUÍS'
            ]
        },
        'rota_b_cenario_2': {
            'origem': [
                'SBPB - PARNAÍBA', 'SBFZ - FORTALEZA', 'SBTE - TERESINA',
                'SBSL - SÃO LUÍS', 'SBTE - TERESINA', 'SBFZ - FORTALEZA',
                'SBPB - PARNAÍBA', 'SBSL - SÃO LUÍS', 'SBTE - TERESINA',
                'SBFZ - FORTALEZA'
            ],
            'destino': [
                'SBFZ - FORTALEZA', 'SBTE - TERESINA', 'SBSL - SÃO LUÍS',
                'SBTE - TERESINA', 'SBFZ - FORTALEZA', 'SBPB - PARNAÍBA',
                'SBTE - TERESINA', 'SBFZ - FORTALEZA', 'SBPB - PARNAÍBA',
                'SBSL - SÃO LUÍS'
            ]
        }
    }

    # Criar os DataFrames a partir dos dados
    rotas = {nome: pd.DataFrame(dados_rotas) for nome, dados_rotas in dados.items()}
    
    return rotas

rotas = criar_rotas()
rota_a_cenario_1 = rotas['rota_a_cenario_1']
rota_a_cenario_2 = rotas['rota_a_cenario_2']
rota_b_cenario_1 = rotas['rota_b_cenario_1']
rota_b_cenario_2 = rotas['rota_b_cenario_2']